In [ ]:
%pip install -r ./requirements.txt

In [10]:
import os
from dotenv import load_dotenv
from art import tprint

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.llm_math.base import LLMMathChain
from langchain_community.utilities import BingSearchAPIWrapper
from langchain.agents import Tool, initialize_agent, tool, AgentType

In [7]:
load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
model_name = "gpt-3.5-turbo-0125"

In [8]:
llm = ChatOpenAI(
    model = model_name,
    temperature = 0.5,
    openai_api_key = openai_api_key
)

llm_math = LLMMathChain.from_llm(llm)
math_tool = Tool(
    name = "Calculator",
    func = llm_math.run,
    description = "Useful tool when you need to answer questions about mathematics."
)

In [9]:
@tool("BingSearch")
def search(search_query: str):
    """
    Useful to search for any information and
    useful when you need to search the internet for any kind of information.
    """

    search = BingSearchAPIWrapper()
    return search.run(search_query)

In [12]:
tools = [search, math_tool]

agent = initialize_agent(
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 10
)

agent.handle_parsing_errors = True

In [ ]:
query = "What's the root over 25? And let me know the capital of Malaysia?"

response = agent(query)

print(response)